## Overview

Playing around with PySpark with Tips.csv dataset on databricks

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

df = spark.read.csv(file_location, header=True, inferSchema=True)

df.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
df.columns

Out[5]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
### Handling Categorical Features

from pyspark.ml.feature import StringIndexer

In [0]:
# indexer=StringIndexer(inputCol="sex",outputCol="sex_indexed")
# df_r=indexer.fit(df).transform(df)
# df_r.show()

In [0]:
indexer=StringIndexer(inputCols=['sex','smoker','day','time'],outputCols=['Sex_Indexed','Smoker_Indexed','Day_Indexed','Time_Indexed'])
df_mod=indexer.fit(df).transform(df)
df_mod.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|Sex_Indexed|Smoker_Indexed|Day_Indexed|Time_Indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [0]:
df_mod.columns

Out[15]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'Sex_Indexed',
 'Smoker_Indexed',
 'Day_Indexed',
 'Time_Indexed']

In [0]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['tip','Sex_Indexed','Smoker_Indexed','Day_Indexed','Time_Indexed','size'],outputCol='Independant Features')
output=featureassembler.transform(df_mod)
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|Sex_Indexed|Smoker_Indexed|Day_Indexed|Time_Indexed|Independant Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,1.0,0.0,1.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[1.66,0.0,0.0,1.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[3.5,0.0,0.0,1.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|[3.31,0.0,0.0,1.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,1.0,0.0,1.0...|
|     25.29|4.71|  Male|    No|S

In [0]:
output.select('Independant Features').show()

+--------------------+
|Independant Features|
+--------------------+
|[1.01,1.0,0.0,1.0...|
|[1.66,0.0,0.0,1.0...|
|[3.5,0.0,0.0,1.0,...|
|[3.31,0.0,0.0,1.0...|
|[3.61,1.0,0.0,1.0...|
|[4.71,0.0,0.0,1.0...|
|[2.0,0.0,0.0,1.0,...|
|[3.12,0.0,0.0,1.0...|
|[1.96,0.0,0.0,1.0...|
|[3.23,0.0,0.0,1.0...|
|[1.71,0.0,0.0,1.0...|
|[5.0,1.0,0.0,1.0,...|
|[1.57,0.0,0.0,1.0...|
|[3.0,0.0,0.0,1.0,...|
|[3.02,1.0,0.0,1.0...|
|[3.92,0.0,0.0,1.0...|
|[1.67,1.0,0.0,1.0...|
|[3.71,0.0,0.0,1.0...|
|[3.5,1.0,0.0,1.0,...|
|(6,[0,5],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [0]:
final_data=output.select('Independant Features','total_bill')
final_data.show()
## total bill is the dependant feature.

+--------------------+----------+
|Independant Features|total_bill|
+--------------------+----------+
|[1.01,1.0,0.0,1.0...|     16.99|
|[1.66,0.0,0.0,1.0...|     10.34|
|[3.5,0.0,0.0,1.0,...|     21.01|
|[3.31,0.0,0.0,1.0...|     23.68|
|[3.61,1.0,0.0,1.0...|     24.59|
|[4.71,0.0,0.0,1.0...|     25.29|
|[2.0,0.0,0.0,1.0,...|      8.77|
|[3.12,0.0,0.0,1.0...|     26.88|
|[1.96,0.0,0.0,1.0...|     15.04|
|[3.23,0.0,0.0,1.0...|     14.78|
|[1.71,0.0,0.0,1.0...|     10.27|
|[5.0,1.0,0.0,1.0,...|     35.26|
|[1.57,0.0,0.0,1.0...|     15.42|
|[3.0,0.0,0.0,1.0,...|     18.43|
|[3.02,1.0,0.0,1.0...|     14.83|
|[3.92,0.0,0.0,1.0...|     21.58|
|[1.67,1.0,0.0,1.0...|     10.33|
|[3.71,0.0,0.0,1.0...|     16.29|
|[3.5,1.0,0.0,1.0,...|     16.97|
|(6,[0,5],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression

train_data,test_data = final_data.randomSplit([0.75, 0.25])
regressor=LinearRegression(featuresCol='Independant Features', labelCol='total_bill')
regressor=regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[27]: DenseVector([2.9422, -0.7287, 2.5222, 0.3287, -2.1351, 3.3182])

In [0]:
regressor.intercept

Out[28]: 2.1196108925786374

In [0]:
## making predictions 

pred=regressor.evaluate(test_data)

In [0]:
pred.predictions.show()

+--------------------+----------+------------------+
|Independant Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,5],[1.25,2.0])|     10.51|12.433719976545518|
|(6,[0,5],[1.45,2.0])|      9.55|13.022151822474473|
|(6,[0,5],[1.97,2.0])|     12.02|14.552074621889751|
| (6,[0,5],[2.0,2.0])|     13.37|14.640339398779098|
| (6,[0,5],[2.0,3.0])|     16.31|17.958544422234553|
|(6,[0,5],[2.01,2.0])|     20.23|14.669760991075542|
|(6,[0,5],[2.72,2.0])|     13.28|16.758694044123335|
|(6,[0,5],[6.73,4.0])|     48.27| 35.19316260190978|
|[1.0,1.0,0.0,0.0,...|      7.25| 7.651300440435567|
|[1.0,1.0,1.0,3.0,...|      5.75|14.477798580510186|
|[1.1,1.0,1.0,0.0,...|      12.9|13.785896883971095|
|[1.32,0.0,0.0,1.0...|      9.68|12.968376995788507|
|[1.44,0.0,1.0,0.0...|      7.74|15.514905727293616|
|[1.5,0.0,1.0,1.0,...|     15.69| 16.02014115424016|
|[1.5,1.0,0.0,0.0,...|     26.41|12.440585078713411|
|[1.61,1.0,1.0,0.0...|     10.59|15.2863980910

In [0]:
pred.rootMeanSquaredError, pred.meanSquaredError

Out[32]: (5.227281210636789, 27.324468855076418)